# Lab1 — PyTorch Foundations for Computer Vision

**Course**: Deep Learning for Image Analysis 

**Class**: M2 IASD App  

**Professor**: Mehyar MLAWEH

---

## Objectives
By the end of this lab, you should be able to:

- Understand how **neurons and layers** are implemented in PyTorch
- Manipulate **tensors** and reason about shapes
- Use **autograd** to compute gradients
- Implement a **training loop** yourself
- Connect theory (neurons, loss, backprop) to actual code

⚠️ This notebook is **intentionally incomplete**.  
Whenever you see **`# TODO`**, you are expected to write code.


**Deadline:** 🗓️ **Saturday, February 7th (23:59)**

## 🤖 A small (honest) note before you start

Let’s be real for a second.

 I know you **can use LLMs (ChatGPT, Copilot, Claude, etc.)** to help you with this lab.  
And yes, **I use them too**, so don’t worry 😄

👉 **You are allowed to use AI tools.**  
But here’s the deal:

- Don’t just **copy–paste** code you don’t understand  
- Take time to **read, question, and modify** what the model gives you  
- If you can solve a block **by yourself, without AI**, that’s excellent 

Remember:

> AI can write code for you, but **only you can understand it** — and understanding is what matters for exams, projects, and real work.

Use these tools **as assistants, not as replacements for thinking**.

---

## 📚 Useful documentation (highly recommended)

You will often find answers faster (and more reliably) by checking the official documentation:

- **PyTorch main documentation**  
  https://pytorch.org/docs/stable/index.html

- **PyTorch tensors**  
  https://pytorch.org/docs/stable/tensors.html

- **Neural network modules (`torch.nn`)**  
  https://pytorch.org/docs/stable/nn.html

- **Loss functions** (`BCEWithLogitsLoss`, CrossEntropy, etc.)  
  https://pytorch.org/docs/stable/nn.html#loss-functions

- **Optimizers** (`SGD`, `Adam`, …)  
  https://pytorch.org/docs/stable/optim.html

If you learn how to **navigate the documentation**, you are already thinking like a real AI engineer 👌

---

In [84]:
# Installation des modules
%pip install numpy matplotlib torch scikit-learn

  Using cached joblib-1.5.3-py3-none-any.whl.metadata (5.5 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 17.0 MB/s  0:00:00 eta 0:00:01
Using cached joblib-1.5.3-py3-none-any.whl (309 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 10.9 MB/s  0:00:01m0:00:0100:01
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [scikit-learn] [scikit-learn]

[notice] A new release of pip is available: 25.2 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## PART I

## 0) Colab setup — GPU check

**Instructions**
1. In Colab: `Runtime → Change runtime type to GPU T4` 
2. Select **GPU**
3. Save and restart runtime

Then run the cell below.


In [ ]:
import torch

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

# TODO: set the device correctly (cuda if available, else cpu)
device = (
        'cuda' if torch.cuda.is_available() 
        else 'mps' if torch.mps.is_available
        else 'cpu'
)

print("Using device:", device)

PyTorch version: 2.10.0
CUDA available: False
Using device: mps


## 1) Imports and reproducibility


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# TODO: fix the random seed for reproducibility
torch.manual_seed(42)

## 2) PyTorch tensors and shapes

Tensors are multi-dimensional arrays that support:
- GPU acceleration
- automatic differentiation

Understanding **shapes** is critical in deep learning.


In [11]:
# Examples
a = torch.tensor([1.0, 2.0, 3.0])
b = torch.randn(4, 5)

print("a shape:", a.shape)
print("b shape:", b.shape)

a shape: torch.Size([3])
b shape: torch.Size([4, 5])


### 🔍 Question (answer inside the markdown)
- How many dimensions does tensor `b` have?
    > `b` is a 2-dimensional tensor, that is a matrix.

- What does each dimension represent conceptually?
    > Conceptually, the first dimension is the number of **rows** and the second one reprensents the number of **columns**. Commonly, each row is an individual and each column is a particular feature (e.g. age, size, ...).

### ✅Tensor operations

Complete the following:

1. Create a tensor `x` of shape `(8, 3)` with random values  
2. Compute:
   - the **mean of each column**
   - the **L2 norm of each row**
3. Normalize `x` **row-wise** using the L2 norm

In [67]:
import numpy as np

# TODO: create x
x = torch.randn(8, 3)

# TODO: column mean
col_mean = x.mean(dim=0) # Mean on the dimension to reduce, here the first dimension to get a tensor of 3 means

# TODO: row-wise L2 norm
row_norm = torch.linalg.norm(x, ord=2, dim=1) # L2-normalization on the rows

# TODO: normalized tensor
x_normalized = col_mean / row_norm.view(-1, 1) # Normalization by row 

print(x.shape, col_mean.shape, row_norm.shape, x_normalized.shape)

torch.Size([8, 3]) torch.Size([3]) torch.Size([8]) torch.Size([8, 3])


## 3) Artificial neuron — from math to code

A neuron computes:

$$
z = \sum_i w_i x_i + b
$$

Then applies an activation function:

$$
y = g(z)
$$

This section connects directly to the theory seen in class.


In [46]:
x = torch.tensor([1.0, -2.0, 3.0])
w = torch.tensor([0.2, 0.4, -0.1])
b = torch.tensor(0.1)

z = torch.sum(x * w) + b
z

tensor(-0.8000)

### Activation functions

1. Implement **ReLU**
2. Implement **Sigmoid**
3. Apply both to `z` and compare the outputs

Which activation preserves negative values?
> By definition, Sigmoid function preserves negative values. Indeed, for any value of $z$, $e^{-z}$ is not zero which implies that $\frac{1}{1 + e^{-z}}$ is nonzero even when $z \le 0$. 

In [74]:
# TODO
def relu(z):
    """Compute relu(x) = max(x, 0)"""
    return torch.max(z, torch.tensor(0))

def sigmoid(z):
    """Compute sigma(x) = 1 / (1 + exp{-z})"""
    return 1 / (1 + torch.exp(-z))

y_relu = relu(z)
y_sigmoid = sigmoid(z)
y_relu, y_sigmoid


(tensor(0.), tensor(0.3100))

## 4) Autograd and gradients

PyTorch uses **automatic differentiation** to compute gradients
using the **chain rule** (backpropagation).


In [75]:
x = torch.tensor([1.0, 2.0, -1.0], requires_grad=True)
w = torch.tensor([0.5, -0.3, 0.8], requires_grad=True)
b = torch.tensor(0.2, requires_grad=True)

z = torch.sum(x * w) + b
loss = (z - 1.0) ** 2

loss.backward()

print("loss:", loss.item())
print("grad w:", w.grad)
print("grad b:", b.grad)

loss: 2.890000104904175
grad w: tensor([-3.4000, -6.8000,  3.4000])
grad b: tensor(-3.4000)


### 🔍 Conceptual question

- If `b.grad > 0`, should `b` increase or decrease after a gradient descent step?
Explain **why** in one sentence.
> A gradient step is defined by $b \leftarrow b - \alpha \nabla f(b)$, so $b$ will decrease after this step. Indeed, as the learning rate is positive, the product of the learning rate with the gradient of $b$ is positive.


## 5) Toy classification dataset

We create a **linearly separable** dataset.

Label rule:
- class = 1 if `x₁ + x₂ + x₃ > 0`
- else class = 0

This mimics a very simple classification problem.


In [146]:
from sklearn import model_selection

n, d = 500, 3

# TODO: generate a dataset of size N=500 with 3 features
X = torch.randn(n,d)
y = (X.sum(dim=-1) > 0).float().view(-1,1)  # shape (N, 1)

# TODO: split into train (80%) and validation (20%)
X_train, X_val, y_train, y_val = model_selection.train_test_split(X, y, train_size=.8, test_size=.2, stratify=y)

## 6) Model definition

We define a small **MLP** (fully-connected network):

`3 → 16 → 8 → 1`

Activation: ReLU  
Output: raw logits (no sigmoid)


In [166]:
class MLP(nn.Module):
    def __init__(self, input_dim=3, output_dim=1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, output_dim)
        )

    def forward(self, x):
        return self.net(x)

# TODO: create model and move it to the GPU 
model = MLP()
model = model.to(device)


###  parameters

1. Compute **by hand** the total number of parameters
> The number of parameters for a layer is : `input_dim x output_dim + output_dim`. The total number of parameters is $(3 \times 16 + 16) + (16 \times 8 + 8) + (8 \times 1 + 1) = 64 + 136 + 9 = 209$

2. Verify your answer using PyTorch


In [167]:
# TODO: count parameters with PyTorch
total_params = sum([parameter.numel() for parameter in model.parameters()])
total_params


209

## 7) Training loop 

You must complete the full training loop:
- forward pass
- loss computation
- backward pass
- optimizer step

Loss: `BCEWithLogitsLoss`
Optimizer: `SGD`


In [168]:
# TODO: move data to device
X_train_d = X_train.to(device)
y_train_d = y_train.to(device)
X_val_d = X_val.to(device)
y_val_d = y_val.to(device)

# Number of epochs
n_epochs = 20

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()

    # TODO: forward
    logits = model.forward(X_train_d)

    # TODO: loss
    loss = criterion(logits, y_train_d)

    # TODO: backward
    loss.backward()

    # TODO: update
    optimizer.step()

    if epoch % 5 == 0:
        print("Epoch", epoch, "| loss =", float(loss))


Epoch 0 | loss = 0.7084853053092957
Epoch 5 | loss = 0.7016672492027283
Epoch 10 | loss = 0.6953663229942322
Epoch 15 | loss = 0.6890119910240173


## 8) Evaluation

1. Apply `sigmoid` to the logits
2. Convert probabilities to predictions
3. Compute **accuracy** on the validation set


In [169]:
# TODO: evaluation
with torch.no_grad():
    logits = model.forward(X_val_d)
    probs = sigmoid(logits)
    preds = (probs >= .5).float()

accuracy = (preds == y_val_d).float().mean()
accuracy

tensor(0.5600, device='mps:0')

## 9) Reflection questions (answer inside the markdown)

1. Why do we **not** apply sigmoid inside the model?
> 

2. What would happen if we removed all ReLU activations?
> If we removed all ReLU activations, the Multi-Layer Perceptron would perform an operation which is equivalent to a linear transformation of the inputs. ReLU activations add nonlinear transformation accross the layers.

3. How does this toy problem relate to image classification?
> This toy problem relate to image classification in the sense that images can be seen as vectors if flattened. And the use of Deep Learning leverages nonlinear transformations in order to recongnize patterns and local structures.

Write short answers (2–3 lines each).


## 10) Bridge to Computer Vision

So far:
- inputs = vectors of size 3
- layers = fully-connected

Next session:
- inputs = images `(B, C, H, W)`
- layers = convolutions
- same training logic

👉 **Architecture changes, learning principles stay the same.**


## Part II — Training on MNIST

Check the next notebook